# Úkol 2: Test hypotézy
*Hypotézy jsou ověřovány s 95% jistotou*

Import knihoven a načtení DataFramu

In [19]:
import pandas as pd
import scipy.stats

df = pd.read_pickle("accidents.pkl.gz")

df

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,-1,2016-01-01,5,55,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,-1,2016-01-01,5,130,1,3,0,2,...,,,,,,-1,-1,,1,PHA
2,002100160003,5,-1,2016-01-01,5,100,1,2,0,2,...,,,,,,-1,-1,,1,PHA
3,002100160004,6,-1,2016-01-01,5,120,9,0,0,2,...,,,,,,-1,-1,,1,PHA
4,002100160005,6,-1,2016-01-01,5,2560,2,0,0,2,...,,,,,,-1,-1,,1,PHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572929,190906210457,6,-1,2021-08-30,1,506,2,0,0,2,...,,2611387,,,Pomalý,560359,-1,GN_V0.1UIR-ADR_410,1,KVK
572930,190906210458,6,-1,2021-08-30,1,752,2,0,0,2,...,,2541503,,Opačnýkesměruúseku,Pomalý,560383,550426,GN_V0.1UIR-ADR_410,1,KVK
572931,190906210459,3,2099,2021-08-30,1,1505,1,3,0,2,...,,,,,,-1,-1,,1,KVK
572932,190906210460,3,2099,2021-08-31,2,1245,0,0,0,1,...,2099,199764,,Opačnýkesměruúseku,Pomalý,560286,553654,GN_V0.1UIR-ADR_410,1,KVK


## Hypotéza 1:
**Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích třetí třídy.**
*K ověření hypotézy je využit χ<sup>2</sup> test*

**H0**: Neexistuje žádná závislost mezi úmrtími při nehodách na silnicích 1. a 3. třídy
**H1**: Existuje závislost mezi úmrtími při nehodách na silnicích 1. a 3. třídy

Vytáhnutí pouze potřebných dat z DataFramu

In [20]:
data = pd.DataFrame()
data[['silnice', 'mrtvi']] = df[['p36', 'p13a']].copy()
# data = data[(data['mrtvi'] > 0) & ((data['silnice'] == 1) | (data['silnice'] == 3))]
data = data[(data['silnice'] == 1) | (data['silnice'] == 3)]
data['nekdo_umrel'] = data['mrtvi'] > 0

data

,silnice,mrtvi,nekdo_umrel
7594,1,0,False
10530,1,0,False
10532,1,0,False
13362,1,0,False
21478,1,0,False
...,...,...,...
572914,3,0,False
572922,3,0,False
572931,3,0,False
572932,3,0,False


Provedení pivotu pro získání údajů o úmrtnosti na silnicích

In [21]:
data_pivot = pd.pivot_table(data, index='nekdo_umrel', columns='silnice', values='nekdo_umrel', aggfunc='count')
# data_crosstab = pd.crosstab(index=data['nekdo_umrel'], columns=data['silnice'], values=data['nekdo_umrel'], aggfunc='count')
data_pivot

silnice,1,3
nekdo_umrel,,
False,78618,73352
True,911,448


Výpočet pomocí χ<sup>2</sup> testu

In [22]:
vysledek = scipy.stats.chi2_contingency(observed=data_pivot)
vysledek[1]

3.5395243450138555e-29

P-hodnota 3,5395e<sup>-29</sup> je nižší než 0,05, proto zamítáme hypotézu H0 na hladině významnosti α=0,05.

Můžeme tedy tvrdit, že existuje závislost mezi pravděpodobností úmrtí na silnicích 1. a 3. třídy, což lze vidět i v tabulce *data_pivot*, kde počet nehod je srovnatelný, ale počet úmrtí je na silnicích první třídy skoro dvojnásobný.

---
# Hypotéza 2:
**Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi**
*K ověření hypotézy je využit Mann-Whitneyův pořadový test*

**H0**: Při nehodách není rozdíl ve škodách na vozidlech značek Audi a Škoda
**H1**: Při nehodách je rozdíl ve škodách na vozidlech značek Audi a Škoda

Vytáhnutí potřebných dat

- značka 2 je AUDI
- značka 39 je ŠKODA

In [23]:
data2 = pd.DataFrame()
data2[['znacka', 'skoda']] = df[['p45a', 'p53']].copy()
data2 = data2[(data2['znacka'] == 2) | (data2['znacka'] == 39)]

data2

,znacka,skoda
0,2,4000
6,39,200
9,39,300
11,39,50
16,39,1200
...,...,...
572911,39,10
572916,39,130
572921,39,100
572923,39,0


Vytvoření sérií pro vozidla značek Škoda a Audi, celková škoda na vozidlech

In [24]:
data2_skoda = data2.loc[data2['znacka'] == 39, 'skoda']
data2_audi = data2.loc[data2['znacka'] == 2, 'skoda']

data2_pivot = pd.pivot_table(data2, columns='znacka', aggfunc='sum')
data2_pivot

znacka,2,39
skoda,6504868,43583126


Počet nehod vozidel značek

In [25]:
print('Audi:\t', len(data2[data2['znacka'] == 2]), '\nŠkoda:\t', len(data2[data2['znacka'] == 39]))

Audi:	 11445 
Škoda:	 118379


Provedení Mann-Whitneyova testu

In [26]:
scipy.stats.mannwhitneyu(data2_skoda, data2_audi)

MannwhitneyuResult(statistic=565845919.0, pvalue=4.921643600739696e-188)

P-hodnota 4,9216e<sup>-188</sup> je mnohem nižší než 0,05, proto zamítáme hypotézu H0 na hladině významnosti α=0,05.

Můžeme tedy tvrdit, že je rozdíl ve škodách na vozidlech u značek Audi a Škoda, což lze vidět i z *data2_pivot* a počtu nehod, kde počet nehod je asi 10x vyšší u vozidel značky Škoda, ale škoda na vozidlech je vyšší pouze 7x.